![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

## Wherobots Inference - Object Detection

This example demonstrates query inference using an object detection model with Wherobots Inference to identify marine infrastruture (offshore wind farms and platforms) in satellite imagery. We will use a machine-learning model from [Satlas](https://satlas.allen.ai/ai) <sup>1</sup> which was trained using imagery from the European Space Agency’s Sentinel-2 satellites.

**Note: This notebook requires the Wherobots Inference functionality to be enabled and a GPU runtime selected in Wherobots Cloud. Please [contact us](https://wherobots.com/contact/) to enable these features.**


### Step 1: Set Up The WherobotsDB Context

Here we configure WherobotsDB to enable access to the necessary cloud object storage buckets with sample data and to enable the WherobotsAI features in WherobotsDB. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from wherobots.inference.data.io import read_raster_table
from sedona.spark import SedonaContext
from pyspark.sql.functions import expr

config = SedonaContext.builder().appName('object-detection-batch-inference')\
    .getOrCreate()

sedona = SedonaContext.create(config)

### 2: Load Satellite Imagery

Next, we load the satellite imagery that we will be running inference over. These GeoTiff images are loaded as *out-db* rasters in WherobotsDB, where each row represents a different scene.

In [ ]:
tif_folder_path = 's3://wherobots-benchmark-prod/data/ml/satlas-offshore-wind-scenes/'
files_df = read_raster_table(tif_folder_path, sedona, limit=1000)
df_raster_input = files_df.withColumn(
        "outdb_raster", expr("RS_FromPath(path)")
    )

In [ ]:
#%%time
df_raster_input.show(truncate=False)

### 3: Run Predictions And Visualize Results

To run predictions we will specify the model we wish to use. Some models are pre-loaded and made available in Wherobots Cloud. We can also load our own models. Predictions can be run using Wherobot's Spatial SQL functions, in this case `RS_DETECT_BBOXES`.

Here we generate 100 predictions using `RS_DETECT_BBOXES`.

In [ ]:
df_raster_input.createOrReplaceTempView("df_raster_input")
model_id = 'marine-satlas-sentinel2'
sedona.sql(f"""
 CREATE TEMP VIEW detect_fields AS (
    SELECT
    outdb_raster,
    RS_DETECT_BBOXES('{model_id}', outdb_raster) AS detect_result
  FROM
    df_raster_input)
""")

predictions_df = sedona.sql(f"""
SELECT
  outdb_raster,
  detect_result.*
FROM detect_fields
""")

predictions_df.cache().show()

predictions_df.createOrReplaceTempView("predictions_df")

Since we ran inference across many country coastlines all over the world, many scenes don't contain wind farms and don't have positive detections. Now that we've generated predictions using our model over our satellite imagery, we can filter the geometries by confidence score and by the integer label representing offshore wind farms, `2`, to locate predicted offshore wind farms. 

In [ ]:
from pyspark.sql.functions import expr, explode, array_contains, col

# Filter the DataFrame
filtered_predictions_df = predictions_df.filter(
    # Filter out rows where none of the 'confidence_scores' are above the threshold
    (expr(f"array_min(transform(confidence_scores, x -> x < .65))") == False) &
    # Filter out rows where 'labels' does not contain the label
    (array_contains(col('labels'), 2))
)

filtered_predictions_df.cache().show()

The final steps are to use `explode` to make each bounding box detection its own row and then to convert our string column representing a geometry into a `GeometryType` column so that we can plot it with `SedonaKepler`.

In [ ]:
# Explode the 'bboxes_wkt' column
df_exploded = filtered_predictions_df.select("outdb_raster", explode(filtered_predictions_df.bboxes_wkt).alias("bboxes_wkt"), "confidence_scores", "labels")

# Apply the 'ST_GeomFromWkt' function
df_exploded = df_exploded.withColumn("geom", expr("ST_GeomFromWkt(bboxes_wkt)")).drop("bboxes_wkt")

In [ ]:
df_exploded.show()

Zoom into the coasts of China or the Netherlands to spot some detected wind farms!

In [ ]:
from sedona.maps.SedonaKepler import SedonaKepler
config = {
    'version': 'v1',
    'config': {
        'mapStyle': {
            'styleType': 'light',
            'topLayerGroups': {},
            'visibleLayerGroups': {},
            'mapStyles': {}
        }
    }
}
map = SedonaKepler.create_map(config=config)

SedonaKepler.add_df(map, df=df_exploded.drop("outdb_raster"), name="Wind Farm Detections")
map

### wherobots.inference Python API

If you prefer python, wherobots.inference offers a module for registering the SQL inference functions as python functions. Below we run the same inference as before with `RS_DETECT_BBOXES`.

In [ ]:
from wherobots.inference.engine.register import create_object_detection_udfs
from pyspark.sql.functions import col
rs_detect, rs_threshold_geoms =  create_object_detection_udfs(batch_size = 1, sedona=sedona)
df = df_raster_input.withColumn("detect_result", rs_detect(model_id, col("outdb_raster"))).select(
                               "outdb_raster",
                               col("detect_result.bboxes_wkt").alias("bboxes_wkt"),
                               col("detect_result.confidence_scores").alias("confidence_scores"),
                               col("detect_result.labels").alias("labels")
                           )
df.show()